# Pytorch 基本用法（一）
本节介绍 Pytorch 的基本用法，涉及的主要是对 `Tensor`，即张量的操作。

## 基本套路
Pytorch 的一大特点在于：任何方法，后缀为 _ 时都会**原地**改变对象的值。

例如 `x.add(y)` 返回的是 `x + y`，但 `x.add_(y)` 就会让 `x` 直接变成 `x + y`。

大部分操作都可以指定输出，只要加入 `out` 参数即可。例如：

In [37]:
import torch
res = torch.empty(5, 3)
x = torch.ones(5, 3)
y = torch.ones(5, 3)
torch.add(x, y, out=res)
print(res)
x.add_(y)                   # 等价于 x += y
print(x)

tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])


对元素的访问方式、切片方式什么的都和 Numpy 没有太大区别。理论上来说，张量可以支持大部分 Numpy 数组的操作。

In [38]:
print(x[0, 0])
print(x[0][0])

tensor(2.)
tensor(2.)


很多地方要用到 `dim` 这个参数。**它从 0 开始计数。**

## 生成张量
生成张量的方式分为两种：一种 `torch.tensor`，一种 `torch.Tensor`。

### 大写的 Tensor
Pytorch 内置多种张量类型，如下所示：

![](pytorch_notes_assets/1.jpg)

可以从列表、序列或者 Numpy 数组生成张量类型。

例如：

In [39]:
a = torch.Tensor(range(1, 2))
b = torch.Tensor([[1, 2, 3], [3, 4, 5]]) 
c = torch.LongTensor([[1, 2, 3], [3, 4, 5]])

可以用它来生成空的张量。

In [40]:
d = torch.LongTensor()
print(d)

tensor([], dtype=torch.int64)


### 小写的 tensor
这是一个用于返回张量的函数，**必须**要有参数 `data`。如：

In [41]:
e = torch.tensor((1, 2, 4))

因此要生成空张量需要写成 `torch.tensor(())`，如果直接写 `torch.tensor()` 会报错。

### 比较
前者更像是一个函数，后者更像是一个构造函数。

**注意：用于产生张量的原始数据的格式应该符合规范**，如下面这个例子：

In [42]:
# torch.Tensor([[1, 2, 3], [3, 4]]) 
"""
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-6-6e20bfadcbab> in <module>
----> 1 torch.Tensor([[1, 2, 3], [3, 4]])

ValueError: expected sequence of length 3 at dim 1 (got 2)
"""
pass

## rand 和 randn
`torch.rand(*size, ...)` 和 `torch.randn(*size, ...)` 都可以产生随机张量，区别在于前者产生的是 $[0, 1)$ 之间的均匀分布随机数，后者产生的是服从标准正态分布的随机数。

例如，`torch.randn(12, 10)` 就可以产生一个大小为 $12 \times 10$ 的随机张量。

## 获取张量的大小
通过 `shape` 属性或者 `size()` 方法可以得到一个张量的大小。返回值是 `torch.Size` 类型，它支持一切元组支持的操作。

In [43]:
print(a.shape)
print(a.size())
print(a.size() + b.size())

torch.Size([1])
torch.Size([1])
torch.Size([1, 2, 3])


## 基本函数列表
下面列出了一些最常用的函数。

![](pytorch_notes_assets/2.jpg)

## 生成索引
Pytorch 可以使用类似于 Numpy 的语法生成索引（切片），但是**改变索引的内容就会改变原有内容**。即生成的索引是**浅拷贝**。

In [44]:
a = b[0, :]
a += 1
print(a)
print(b[0, :])

tensor([2., 3., 4.])
tensor([2., 3., 4.])


## 高级选择函数
下面介绍一下比较高级的选择函数（解释和例子来自官方文档），在抽取数据集的时候经常用到。

![](pytorch_notes_assets/3.jpg)

1. `torch.index_select(input, dim, index, out=None)`。沿着指定维度对输入进行切片，取 `index` 中指定的相应项（`index` 为一个 `LongTensor`），然后返回到一个新的张量，返回的张量与原始张量有相同的维度（在指定轴上）。**注意：返回的张量不与原始张量共享内存空间。**

In [45]:
x = torch.randn(3, 4)
indices = torch.LongTensor([0, 2])
print(torch.index_select(x, 0, indices))
print(torch.index_select(x, 1, indices))

tensor([[ 0.9628, -0.0111, -0.1045,  0.1219],
        [-0.1175, -1.2547, -0.0277,  0.7983]])
tensor([[ 0.9628, -0.1045],
        [ 1.9193,  0.7264],
        [-0.1175, -0.0277]])


2. `torch.masked_select(input, mask, out=None)`。根据掩码张量 `mask` 中的二元值，取输入张量中的指定项，将取值返回到一个新的**一维**张量。张量 `mask` 须跟 `input` 张量有相同数量的元素数目，但形状或维度不需要相同。**注意：返回的张量不与原始张量共享内存空间。**

In [46]:
x = torch.randn(3, 4)
indices = x > 0        # 这里得到的是一个 BoolTensor
print(indices)
print(torch.masked_select(x, indices))

tensor([[ True,  True,  True, False],
        [ True, False,  True,  True],
        [ True,  True,  True,  True]])
tensor([1.9386, 0.1998, 1.6955, 0.3264, 0.9476, 0.8980, 0.1613, 1.6387, 0.7990,
        0.5583])


3. `torch.nonzero(input, out=None)`。返回 `input` 中所有非零元素的坐标，一行一个。

In [47]:
print(torch.nonzero(torch.Tensor([[0.6, 0.0, 0.0, 0.0],
                             [0.0, 0.4, 0.0, 0.0],
                            [0.0, 0.0, 1.2, 0.0],
                             [0.0, 0.0, 0.0,-0.4]])))

tensor([[0, 0],
        [1, 1],
        [2, 2],
        [3, 3]])


4. `torch.gather(input, dim, index, out=None)`。沿给定轴 `dim`，将输入索引张量 `index` 指定位置的值进行聚合。这里 `index` 的大小应当和 `input` 相同。

In [48]:
"""
以三维张量为例：
out[i][j][k] = tensor[index[i][j][k]][j][k]  # dim=0
out[i][j][k] = tensor[i][index[i][j][k]][k]  # dim=1
out[i][j][k] = tensor[i][j][index[i][j][k]]  # dim=2
"""
t = torch.Tensor([[1,2],[3,4]])
torch.gather(t, 1, torch.LongTensor([[0,0],[1,0]]))

tensor([[1., 1.],
        [4., 3.]])

## view 方法
`view()` 方法是一个比较神奇的方法，它和 Numpy 的 `reshape()` 比较相似，可以用来改变张量的分布。

可以在恰当的位置传入 `-1` 作为参数以实现自动推断要转换成的张量的分布。

In [49]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
b = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]])
print(a.view(-1))
print(b.view(-1, 2))

tensor([1, 2, 3, 4, 5, 6])
tensor([[1, 2],
        [3, 4],
        [5, 6],
        [1, 2],
        [3, 4],
        [5, 6]])


理论上来说，最多只能出现一次 `-1`，不然是无法定义的。同时要转换成的分布也应当合法，不能出现多余的元素。

In [50]:
# print(b.view(-1, 5))
"""
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
 in 
----> 1 print(b.view(-1, 5))

RuntimeError: shape '[-1, 5]' is invalid for input of size 12
"""
pass

需要注意，**`view()` 和索引差不多，返回的只是原来数据的引用，而并不是新的一份数据。因此改变原数据会影响 `view()` 的结果，反之亦然。**

## 生成一个副本
可以看出，引用是一个比较麻烦的问题。因此如果需要数据独立，应当先对原始数据克隆一份之后再进行操作。

`clone()` 方法可以返回一个原数据的拷贝。

In [51]:
a_clone = a.clone().view(-1)
a_clone[0] = -1
print(a_clone)
print(a)

tensor([-1,  2,  3,  4,  5,  6])
tensor([[1, 2, 3],
        [4, 5, 6]])


## 标量抽取
使用 `item()` 方法可以获得一个零维张量的内部数据。

In [52]:
a = torch.tensor([1, 2])
print(a[0])
print(a[0].item())

tensor(1)
1


## 广播运算
Pytorch 的广播运算有时候非常离谱，总体上应该遵循的是**最小公倍数原则**。例如一个行向量和一个列向量加起来得到的是一个矩阵。

In [53]:
a = torch.zeros(1, 5)
b = torch.zeros(3, 1)
print(a + b)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


## 和 Numpy 交互
Pytorch 和 Numpy 之间可以方便的交换类型，即前者的张量很容易转成后者的多维数组，反之亦然。

使用 `numpy()` 方法将张量转换为 Numpy 数组，使用 `torch.from_numpy()` 函数将 Numpy 数组转换成为张量。

需要注意：**直接转换时，这两个对象共用一片内存**。如果要让内存上无关，需要使用各自的构建函数，即 `numpy.array()` 和 `torch.tensor()`**（注意不是 `torch.Tensor()`！）**。

In [54]:
import numpy as np

a = torch.ones(5)
b = a.numpy()
print(a, b)

a += 1
print(a, b)
b += 1
print(a, b)

b = np.array(a)
b += 1
print(a, b)
c = torch.tensor(b)
c += 1
print(b, c)

tensor([1., 1., 1., 1., 1.])[1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.])[2. 2. 2. 2. 2.]
tensor([3., 3., 3., 3., 3.])[3. 3. 3. 3. 3.]
tensor([3., 3., 3., 3., 3.])[4. 4. 4. 4. 4.]
[4. 4. 4. 4. 4.]tensor([5., 5., 5., 5., 5.])
